In [1]:
import pandas as pd
from itertools import cycle
import os

In [2]:
caminho = os.path.join('..', '..', 'db_softtek') 
path_arq = os.path.abspath(caminho) 
tbl_ = '\\modelo\\Modelo_score_train.xlsx'

In [3]:
#Tabela de treino:
tbl_train = pd.read_excel(path_arq+tbl_)
analistas_df = tbl_train

In [4]:
#tabela novas demandas:

tbl_test = pd.read_excel(path_arq+'\\cubo_anl\\tbl_full_analitica.xlsx')
novos_chamados_df = tbl_test[['CHAMADO','DT_ABERTURA_CHAMADO','TIPO_CHAMADO','MODULO_CHAMADO','COMPLEXIDADE']].drop_duplicates(keep='first')

In [9]:
# Agrupar analistas por módulo e ordenar por rank
analistas_por_modulo = analistas_df.sort_values(by=['MODULO_CHAMADO', 'rank']).groupby('MODULO_CHAMADO')

# Criar iteradores rotativos para cada módulo
iteradores = {modulo: cycle(grupo['IS'].tolist()) for modulo, grupo in analistas_por_modulo}

# Função para distribuir chamados
def distribuir_chamados(novos_chamados_df, iteradores):
    alocacao = []
    for _, chamado in novos_chamados_df.iterrows():
        modulo = chamado['MODULO_CHAMADO']
        if modulo in iteradores:
            analista = next(iteradores[modulo])
            alocacao.append(analista)
        else:
            alocacao.append(None)  # Caso não haja analistas para o módulo
    return alocacao



# Distribuir os novos chamados
novos_chamados_df['IS'] = distribuir_chamados(novos_chamados_df[['TIPO_CHAMADO','MODULO_CHAMADO','COMPLEXIDADE']], iteradores)

# Exibir a distribuição
novos_chamados_df.sort_values(['TIPO_CHAMADO','MODULO_CHAMADO']).ffill()
novos_chamados_df['ANOMES_ABERTURA'] = novos_chamados_df['DT_ABERTURA_CHAMADO'].str[0:4] + novos_chamados_df['DT_ABERTURA_CHAMADO'].str[5:7]

In [10]:
novos_chamados_df.to_excel(path_arq+'\\modelo\\alocacao_recursos.xlsx', index=False)

novos_chamados_df.sort_values(['TIPO_CHAMADO','MODULO_CHAMADO']).ffill().reset_index().drop(columns='index').to_json(path_arq+'\\modelo\\cubo3_alocacao_recursos.json', orient='records', indent=4)